In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox') #sandbox 사용 X
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_path = 'C:/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)

In [ ]:
def Crawling(input_num):
    try :
        url = f'https://www.crowdpic.net/view/contest/contest_detail.php?n={input_num}'

        driver.get(url)
        soup = bs(driver.page_source, 'html.parser')

        # 제목
        title = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.contest-title-wrap > div.contest-title').text
        
        # 세부설명
        explain = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.contest-explain').text
        
        # 만료기간
        end_period = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.period > div.end-date > span').text
        end_period = end_period[end_period.index('~')+2:] # 목표 수량만 출력

        try :
            # 현재 업로드 수
            upload_count = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.uploaded-count > span').text
            upload_count = int(upload_count.replace(',','')) # 콤마제거 후 정수로 받음
            
            # 목표 수량
            goal_count = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.goal-count').text
            goal_count = goal_count[goal_count.index('/')+2:] # 목표 수량만 출력
            goal_count = int(goal_count.replace(',','')) # 콤마제거 후 정수로 받음
            
            # 총 상금
            total_reward = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.total-reward > span').text[:-1] # 통화단위 제거
            total_reward = int(total_reward.replace(',','')) # 콤마제거 후 정수로 받음
            
            # 개당 리워드
            per_price = total_reward // goal_count

            CloudPick = {'seq': input_num, 'title': title, 'end_period':end_period, 'upload_count':upload_count, 'goal_count':goal_count, 'total_reward':total_reward, 'per_price':per_price, 'explain':explain }
            CloudPick_df = pd.DataFrame(CloudPick, index=[0])
            CloudPick_df.set_index('seq', inplace=True)

        except:
            # 총 상금
            total_reward = soup.select_one('#contest-detail-container > div.contest-contents-wrap > div.total-reward > span').text[:-1] # 통화단위 제거
            total_reward = int(total_reward.replace(',','')) # 콤마제거 후 정수로 받음
            
            CloudPick = {'seq': input_num, 'title': title, 'end_period':end_period, 'upload_count': None, 'goal_count':None, 'total_reward':total_reward, 'per_price':None, 'explain':explain }
            CloudPick_df = pd.DataFrame(CloudPick, index=[0])
            CloudPick_df.set_index('seq', inplace=True)
            
        return CloudPick_df

    except:
        pass

In [ ]:
# 전체 크롤링 (이전 내역까지)
df = pd.concat([Crawling(x) for x in range(1,625)])

# 설명 컬럼 제거
df_drop_explain = df.drop(['explain'], axis=1)
df_drop_explain

# 저장
df.to_csv("크라우드픽_All.csv", encoding='utf-8-sig')
df_drop_explain.to_csv("크라우드픽_drop_explain.csv", encoding='utf-8-sig')

In [ ]:
# 이미지 추출

from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import requests
import urllib.request
import os

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox') #sandbox 사용 X
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_path = 'C:/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)

In [ ]:
# 이미지 추출

# 이미지 폴더 만들기
os.makedirs('Cloudpick_img')

for i in range(1,100):  # 페이지 SEQ 시작, 끝 값
    for j in range(1,10): # 페이지 안에 추출할 이미지 갯수
        try:
            # 사이트 주소 -> SEQ 규칙 필요
            url = f'https://www.crowdpic.net/view/contest/contest_detail.php?n={i}'

            driver.get(url)
                        
            soup = bs(driver.page_source, 'html.parser')
                    
            # 해당 페이지에서 이미지 주소로 접근하는 selector (F12에서 Copy Selector하면 됨 !!)
            res = soup.select_one(f'#contest-detail-container > div.contest-contents-wrap > div.contest-example-wrap > div.pagination-container > div > div:nth-of-type({j})')
                    
            img = res.find('img')['src']
            
            urllib.request.urlretrieve(img, f"Cloudpick_img/cloudpick_{i}_{j}.jpg")
            
        
        except:
            print(i, j, "fail")        
            pass